In [1]:
from IPython.display import clear_output
from itertools import permutations, product
import time
from functools import cache
SLEEP_TIME = 0.05

In [2]:
with open('input.txt', 'r') as fl:
    codes = [code.strip() for code in fl.readlines()]

In [3]:
codes

['879A', '508A', '463A', '593A', '189A']

In [4]:
# helper functions
def get_all_paths(loc1, loc2, bad=None):
    #gets all paths on a grid
    di, dj = loc2[0] - loc1[0], loc2[1] - loc1[1]
    di_sgn, dj_sgn = -1 if di < 0 else 1, -1 if dj < 0 else 1
    pth = list((di_sgn, 0) for _i in range(di_sgn, di + di_sgn, di_sgn)) + \
           list((0, dj_sgn) for _j in range(dj_sgn, dj + dj_sgn, dj_sgn))
    pths = list(set(permutations(pth)))
    moves = []
    for pth in pths:
        if bad:
            good = True
            pos = loc1
            for df in pth:
                pos = pos[0]+df[0], pos[1]+df[1]
                if pos == bad:
                    good = False
                    break
            if not good:
                continue
        moves.append(''.join(rev_directions[d] for d in pth))
    return moves

def get_keypad_paths(loc1, loc2):
    return get_all_paths(loc1, loc2, bad=(3,0))

def get_arrow_paths(loc1, loc2):
    return get_all_paths(loc1, loc2, bad=(0,0))

# get all paths from stacked up arrow layers
@cache
def stacked_arrow_paths(a1, a2, depth=1, delim=''):
    loc1, loc2 = rev_arrow_lookup[a1], rev_arrow_lookup[a2] 
    paths = [path + 'A' for path in get_arrow_paths(loc1, loc2)]
    if depth > 1:
        new_paths = []
        for path in paths:
            path = 'A' + path
            new_paths.extend(
                list(delim.join(_p) 
                     for _p in product(
                        *[stacked_arrow_paths(path[i],path[i+1], depth=depth-1) 
                            for i in range(len(path)-1)]
                    )))
        paths = new_paths
    return paths

# just get the shortest and cache it. 
@cache
def shortest_arrow_path(a1, a2, depth=1):
    paths = stacked_arrow_paths(a1, a2, depth)
    min_len = min(len(path) for path in paths)
    for path in paths:
        if len(path) == min_len:
            return path

# string together a sequence of numpad things
def num_path(path, arrow_depth=2):
    path = 'A' + path
    out = ''
    for i in range(len(path)-1):
        out += shortest_arrow_path(path[i], path[i+1], arrow_depth)
    return out

# just put it all together
def get_shortest_numpath(target, arrow_depth=2):
    # step 1: get all the arrow paths of the input
    rev_num_lookup = {v:k for k,v in NumberPad().mapping.items()}
    target_locs = [(3,2)] + list(rev_num_lookup[s] for s in target)
    # step 2: get all possible paths through the code on numpad
    _p = list(product(*tuple(get_keypad_paths(target_locs[i],target_locs[i+1]) 
                      for i in range(len(target_locs)-1) )))
    num_paths = ['A'.join(p) + 'A' for p in _p]
    paths = [num_path(path, arrow_depth) for path in num_paths]    
    # just grab the shortest
    min_len = min(len(path) for path in paths)
    for path in paths:
        if len(path) == min_len:
            return path

In [5]:
# I think I'm going to code the "game" to illustrate and get
# intuition
directions = {'<':(0,-1), '^':(-1,0), 'v':(1,0), '>':(0,1)}
rev_directions = {d:v for v, d in directions.items()}

class Input:
    def __init__(self, mapping):
        self.mapping = mapping.copy()
        self.n = max(p[0] for p in self.mapping) + 1
        self.m = max(p[1] for p in self.mapping) + 1
        
    def activate(self, pos):
        return self.mapping[pos]
    
    def display(self, star=None, header=None):
        s = '\n' if header is None else header
        for i in range(self.n):
            for j in range(self.m):
                if (i,j) in star:
                    s += star[(i,j)]
                elif (i,j) in self.mapping:
                    s += self.mapping[(i,j)]
                else:
                    s += 'x'
                s += '  '
            s += '\n'
        print(s)
                    
        
    
class NumberPad(Input):
    def __init__(self):
        mapping = {(0,0):'7', (0,1):'8', (0,2):'9',
                   (1,0):'4', (1,1):'5', (1,2):'6',
                   (2,0):'1', (2,1):'2', (2,2):'3',
                              (3,1):'0', (3,2):'A'}
        super().__init__(mapping=mapping)
rev_num_lookup = {v:k for k,v in NumberPad().mapping.items()}
        
        
class ArrowPad(Input):
    def __init__(self):
        mapping = {           (0,1):'^', (0,2):'A',
                   (1,0):'<', (1,1):'v', (1,2):'>'}
        super().__init__(mapping=mapping)
rev_arrow_lookup = {v:k for k,v in ArrowPad().mapping.items()}        
        
        
class Robot:
    def __init__(self, input_pad):
        self.input_pad = input_pad
        self.last_move = None
        self.state = list([p for p, v in self.input_pad.mapping.items() if v == 'A'])[0]
        
    def press_button(self):
        self.last_move = 'A'
        return self.input_pad.activate(self.state)
    
    def move_arm(self, direction):
        self.last_move = direction
        i, j = self.state
        di, dj = directions[direction]
        self.state = (i + di, j + dj)
        return None
    
    def display(self, header = 'robot:\n'):
        star = {self.state:'#'} if self.last_move == 'A' else {self.state:'*'}
        self.input_pad.display(header=header, star=star)
        
class Game:
    def __init__(self, robots):
        self.robots = robots
        self.inputs = ''
        
    def next_action(self, key):
        self.inputs += key
        for robot in self.robots:
            out = None
            if key == 'A':
                out = robot.press_button()
            elif key in directions:
                out = robot.move_arm(key)
            else:
                robot.last_move = None
            key = out
        return out
    
    def display(self):
        for robot in self.robots:
            robot.display()
            
    def run_moves(self, moves, verbose=False):
        if verbose:
            self.display()
            clear_output(wait = True)
            
        p = ''
        for s in moves:
            out = game.next_action(s)   
            if out:
                p += out
            if verbose:
                print('INPUT: ', s)
                game.display()

                print('OUTPUT: ', p)
                clear_output(wait = True)
                time.sleep(SLEEP_TIME)
        return p
                

In [6]:
target = codes[0]
game = Game([Robot(ArrowPad()), Robot(ArrowPad()), Robot(NumberPad())])
game.run_moves(get_shortest_numpath(target),
               verbose=False)

'879A'

In [7]:
total = 0
for code in codes:
    len_path = len(get_shortest_numpath(code))
    total += len_path * int(code.replace('A', ''))
total

188384

In [8]:
# part 2

In [9]:
# get all paths from stacked up arrow layers
@cache
def all_arrow_paths(path):
    locs = [rev_arrow_lookup[a] for a in path]
    paths = [[path + 'A' for path in get_arrow_paths(locs[i], locs[i+1])]
             for i in range(len(locs)-1)]
    return list(map(lambda l: ''.join(l), product(*paths))) 

# just get the shortest and cache it. 
 
PATH_CACHE = {}
def reset_cache():
    import gc
    global PATH_CACHE 
    PATH_CACHE = {}
    gc.collect()
#@cache
def shortest_arrow_path_words(arrows, depth=1, ret_len = True, max_cache_depth = 100):
    if arrows == 'A':
        if ret_len:
            return 1
        else:
            return 'A'
    if (arrows, depth, ret_len) in PATH_CACHE:
        return PATH_CACHE[(arrows, depth, ret_len)]
    
    paths = all_arrow_paths('A' + arrows)
    if depth > 1:
        new_paths = []
        for path in paths:
            # words can be solved independently
            words = [word + 'A' for word in path.split('A')[:-1]]
            #print(path, words)
            new_path = 0 if ret_len else ''
            for word in words:
                # recursion where only the single words will be messed with
                new_path += shortest_arrow_path_words(word, depth=depth-1, ret_len=ret_len,
                                                      max_cache_depth = max_cache_depth)
            new_paths.append(new_path)
        paths = new_paths
        
    # return the shortest path or length
    if ret_len:
        if depth == 1:
            min_len = min(len(path) for path in paths)   
        else:
            min_len = min(paths)
        if depth <= max_cache_depth:
            PATH_CACHE[(arrows, depth, ret_len)] = min_len
        return min_len
    else:
        min_len = min(len(path) for path in paths)
        for path in paths:
            # find that path
            if len(path) == min_len:
                if depth <= max_cache_depth:
                    PATH_CACHE[(arrows, depth, ret_len)] = path
                return path

# just put it all together
def get_shortest_numpath_v2(target, arrow_depth=2, ret_len=False):
    # step 1: get all the arrow paths of the input
    rev_num_lookup = {v:k for k,v in NumberPad().mapping.items()}
    target_locs = [(3,2)] + list(rev_num_lookup[s] for s in target)
    # step 2: get all possible paths through the code on numpad
    _p = list(product(*tuple(get_keypad_paths(target_locs[i],target_locs[i+1]) 
                      for i in range(len(target_locs)-1) )))
    num_paths = ['A'.join(p) + 'A' for p in _p]
    paths = [shortest_arrow_path_words(path, arrow_depth, ret_len=ret_len) 
             for path in num_paths]    
    # just grab the shortest
    if ret_len:
        return min(paths)
    else:
        min_len = min(len(path) for path in paths)
        for path in paths:
            if len(path) == min_len:
                return path

In [10]:
target = codes[0]
game = Game([Robot(ArrowPad()) for i in range(10)] + [Robot(NumberPad())])
game.run_moves(get_shortest_numpath_v2(target, arrow_depth=10),
               verbose=False)

'879A'

In [11]:
total = 0
for code in codes:
    len_path = get_shortest_numpath_v2(code, arrow_depth=25, ret_len=True)
    total += len_path * int(code.replace('A', ''))
total

232389969568832